In [1]:
import pandas as pd 
import numpy as np
import sqlite3
conn = sqlite3.connect('CourseData2.db')
c = conn.cursor()

In [2]:
%load_ext sql

In [3]:
%%sql
sqlite:///CourseData2.db

'Connected: @CourseData2.db'

In [4]:
%%sql
--Dropping tables for when we need to restart and clear CourseData database-- 
--Drop ERD tables--
DROP TABLE IF EXISTS SECTION;
DROP TABLE IF EXISTS CATALOG;
DROP TABLE IF EXISTS INSTRUCTOR;
DROP TABLE IF EXISTS COURSE_MEETING;
DROP TABLE IF EXISTS PROGRAM;

--Drop import tables--
DROP TABLE IF EXISTS import_course;
DROP TABLE IF EXISTS import_course_meeting;
DROP TABLE IF EXISTS import_course_catalog;

 * sqlite:///CourseData2.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Create Tables (Import and ERD) ##

In [5]:
%%sql
--Creating import_catalog table-- 

CREATE TABLE 'import_course_catalog' (
    program_code TEXT NOT NULL,
    program_name VARCHAR NOT NULL,
    catalog_id VARCHAR NOT NULL,
    course_title VARCHAR NOT NULL, 
    credits VARCHAR NOT NULL,
    prereqs VARCHAR, 
    coreqs VARCHAR,
    fees VARCHAR,
    attributes VARCHAR,
    description VARCHAR 
);

 * sqlite:///CourseData2.db
Done.


[]

In [6]:
%%sql
--Creating import_course_meeting table-- 

CREATE TABLE 'import_course_meeting' (
    term VARCHAR NOT NULL,
    crn INTEGER NOT NULL,
    location VARCHAR NOT NULL,
    day TEXT NOT NULL,
    start VARCHAR NOT NULL,
    end VARCHAR NOT NULL 
);

 * sqlite:///CourseData2.db
Done.


[]

In [7]:
%%sql
--Creating import_course table-- 

CREATE TABLE 'import_course' (
    term VARCHAR NOT NULL,
    crn INTEGER NOT NULL,
    catalog_id VARCHAR NOT NULL,
    section VARCHAR NOT NULL,
    credits VARCHAR,
    title VARCHAR, 
    meetings VARCHAR,
    timecodes VARCHAR, 
    primary_instructor TEXT,
    cap INTEGER, 
    act INTEGER, 
    rem INTEGER 
);

 * sqlite:///CourseData2.db
Done.


[]

In [8]:
%%sql
-- Creating table INSTRUCTOR with surrogate primary key-- 

CREATE TABLE 'INSTRUCTOR' (
    InID INTEGER NOT NULL PRIMARY KEY,
    Name TEXT NOT NULL
);


--Creating table PROGRAM with PID as the primary key-- 
CREATE TABLE 'PROGRAM'(
    PID INTEGER PRIMARY KEY,
    PName TEXT NOT NULL,
    PCode TEXT NOT NULL
);

--Creating table CATALOG with CatID as the primary key-- 

CREATE TABLE 'CATALOG'(
    CatClass INTEGER PRIMARY KEY,
    CatID VARCHAR,
    Title VARCHAR NOT NULL,
    Description TEXT,
    Credits TEXT NOT NULL,
    PID INTEGER NOT NULL,
    PREREQS TEXT,
    COREQS TEXT,
    FEES INTEGER,
    Attribute TEXT,
    FOREIGN KEY (PID) REFERENCES PROGRAM(PID)
    
);

--Creating table SECTION with surrogate primary key-- 

CREATE TABLE 'SECTION'(
    SID INTEGER NOT NULL PRIMARY KEY,
    CRN INTEGER NOT NULL,
    Term TEXT NOT NULL,
    Section VARCHAR NOT NULL,
    Cap INTEGER,
    Act INTEGER,
    Rem INTEGER,
    CatID VARCHAR,
    InID INTEGER,
    FOREIGN KEY (InID) REFERENCES INSTRUCTOR(InID),
    FOREIGN KEY (CatID) REFERENCES Catalog(CatID)
);


--Creating table COURSE_MEETING with MID as the Primary Key-- 

CREATE TABLE 'COURSE_MEETING' (
    MID INTEGER PRIMARY KEY,
    CRN INTEGER NOT NULL,
    Term TEXT NOT NULL,
    Location VARCHAR NOT NULL,
    Start TEXT, 
    End TEXT, 
    DAY TEXT,
    FOREIGN KEY (CRN) REFERENCES SECTION(CRN),
    FOREIGN KEY (TERM) REFERENCES SECTION(TERM)
);

 * sqlite:///CourseData2.db
Done.
Done.
Done.
Done.
Done.


[]

## Populate the Import Tables with the Data from the csv files ##

**Creating dataframes for the import course**

In [9]:
## populating import_course table
fall = ['SourceData/Fall2014/courses.csv','SourceData/Fall2015/courses.csv', 'SourceData/Fall2016/courses.csv',
        'SourceData/Fall2017/courses.csv','SourceData/Fall2018/courses.csv']
dff0 = pd.read_csv(fall[0])
dff1 = pd.read_csv(fall[1])
dff2 = pd.read_csv(fall[2])    
dff3 = pd.read_csv(fall[3])
dff4 = pd.read_csv(fall[4])

In [10]:
df_fall = pd.concat([dff0, dff1, dff2, dff3, dff4], axis=0)
df_fall.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
1381,Fall2015,75369,SP 0110,C,3.000,Elementary Spanish I,"[{'days': 'MWR', 'times': '0530pm-0620pm', 'da...",['MWR 0530pm-0620pm 09/01-12/07 DMH 348'],Christopher J. Poulos,18,14,4
455,Fall2015,75030,EN 0011,S,3.000,Texts and Contexts I: Writing as Craft and Inq...,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dat...",['TF 1230pm-0145pm 09/01-12/07 CNS 301'],Frank Moliterno,19,19,0
406,Fall2014,73276,ECE 0508,01,3.000,Engineering Entrepreneurship,"[{'days': 'U', 'times': '0100pm-0330pm', 'date...",['U 0100pm-0330pm 09/07-12/14 DSB 110B'],Douglas A. Lyon,60,32,28
590,Fall2014,72693,FTM 0306,A,3.000,"Internship in Film, Television, and Media Arts",[],[],Mark P. Scalese,0,5,-5
651,Fall2014,70155,FE 0001,N,0.000,First Yr Experience-Leadership,"[{'days': 'T', 'times': '1000am-1050am', 'date...",['T 1000am-1050am 09/02-12/08 CNS 301'],Kamala Kiem,27,27,0


In [11]:
df_fall.describe()

,crn,cap,act,rem
count,7486.000000,7486.00000,7486.000000,7486.000000
mean,74523.769169,18.39487,16.310446,2.219076
std,2512.545835,14.41373,9.863494,17.175757
min,70001.000000,0.00000,-1.000000,-75.000000
25%,72213.000000,12.00000,8.000000,0.000000
50%,74848.500000,19.00000,16.000000,1.000000
75%,76698.750000,25.00000,24.000000,5.000000
max,79215.000000,500.00000,90.000000,999.000000


In [12]:
spring = ['SourceData/Spring2015/courses.csv','SourceData/Spring2016/courses.csv', 'SourceData/Spring2017/courses.csv',
        'SourceData/Spring2018/courses.csv','SourceData/Spring2019/courses.csv', 'SourceData/SpringBreak2017/courses.csv']
dfs0 = pd.read_csv(spring[0])
dfs1 = pd.read_csv(spring[1])
dfs2 = pd.read_csv(spring[2])    
dfs3 = pd.read_csv(spring[3])
dfs4 = pd.read_csv(spring[4])
dfs5 = pd.read_csv(spring[5])

In [13]:
df_spring = pd.concat([dfs0,dfs1,dfs2,dfs3,dfs4,dfs5], axis=0)
df_spring.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
1331,Spring2019,37040,SO 0011,01,3.000,Introduction to Sociology,"[{'days': 'TF', 'times': '0930am-1045am', 'dat...",['TF 0930am-1045am 01/22-04/30 CNS 202'],Belinda A. Quartey,30,30,0
685,Spring2015,31991,HR 0202,A,3.000,Honors Seminar: Water and Security,"[{'days': 'M', 'times': '0600pm-0830pm', 'date...",['M 0600pm-0830pm 01/20-04/30 DMH 131'],Gita Rajan,0,16,-16
179,Spring2015,34070,BI 0381C,B,3.000,Seminar: Bacterial Pathogenesis,"[{'days': 'TF', 'times': '0930am-1045am', 'dat...",['TF 0930am-1045am 01/20-04/30 BNW 469'],Olivia T. Harriott,12,7,5
281,Spring2016,38303,CR 0245L,B,1.000,Digital Design I Lab,"[{'days': 'W', 'times': '0200pm-0400pm', 'date...",['W 0200pm-0400pm 01/20-05/03 BNW 133'],James P. Cavallo,15,16,-1
131,Spring2019,37335,BI 0151L,02,0.000,Elements of Microbiology Lab,"[{'days': 'T', 'times': '0930am-1220pm', 'date...",['T 0930am-1220pm 01/22-04/30 BNW 245'],Christine R. Rodriguez,16,16,0


In [14]:
df_spring.describe()

,crn,cap,act,rem
count,7122.000000,7122.000000,7122.000000,7122.000000
mean,35212.001966,18.123982,15.658804,2.465178
std,2618.670948,10.876640,10.342885,6.497100
min,31001.000000,0.000000,0.000000,-47.000000
25%,32773.250000,11.000000,7.000000,0.000000
50%,34835.000000,19.000000,16.000000,1.000000
75%,37693.750000,25.000000,24.000000,4.000000
max,41702.000000,100.000000,90.000000,89.000000


In [15]:
summer = ['SourceData/Summer2015/courses.csv', 'SourceData/Summer2016/courses.csv',
        'SourceData/Summer2017/courses.csv','SourceData/Summer2018/courses.csv']
dfsm0 = pd.read_csv(summer[0])
dfsm1 = pd.read_csv(summer[1])
dfsm2 = pd.read_csv(summer[2])    
dfsm3 = pd.read_csv(summer[3])

In [16]:
df_summer = pd.concat([dfsm0,dfsm1,dfsm2,dfsm3], axis=0)
df_summer.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
44,Summer2018,58366,BB 0138,01,0.000,Retirement Planning and Employer-Sponsored Ben...,"[{'days': 'M', 'times': '0630pm-1000pm', 'date...",['M 0630pm-1000pm 06/11-08/20 DSB 2109A'],Paul A. Rivera,15,4,11
81,Summer2018,58382,ED 0511,01,3.000,Educating for Social Responsibility and Civic ...,"[{'days': 'TR', 'times': '0400pm-0800pm', 'dat...",['TR 0400pm-0800pm 06/05-07/03 CNS 9'],Terri L. Germain-Williams,22,20,2
307,Summer2016,56481,SL 0590,01,3.000,Sixth Year Certificate Advanced Research Pract...,[],[],Anne E Campbell,2,3,-1
57,Summer2018,58452,CO 0397,MP,3.000,Independent Study,[],[],Michael P. Pagano,0,1,-1
258,Summer2015,55037,PY 0101,01,3.000,General Psychology,[],[],Judith Primavera,20,11,9


In [17]:
df_summer.describe()

,crn,cap,act,rem
count,1224.000000,1224.000000,1224.000000,1224.000000
mean,56704.305556,14.843954,10.913399,3.928922
std,1143.898206,10.771969,9.124774,6.881239
min,55001.000000,0.000000,-1.000000,-24.000000
25%,55382.750000,6.000000,5.000000,0.000000
50%,56517.500000,16.000000,9.000000,3.000000
75%,58008.500000,20.000000,15.000000,9.000000
max,58477.000000,60.000000,59.000000,24.000000


In [18]:
winter = ['SourceData/Winter2015/courses.csv', 'SourceData/Winter2016/courses.csv',
        'SourceData/Winter2017/courses.csv','SourceData/Winter2018/courses.csv']
dfw0 = pd.read_csv(winter[0])
dfw1 = pd.read_csv(winter[1])
dfw2 = pd.read_csv(winter[2])    
dfw3 = pd.read_csv(winter[3])

In [19]:
df_winter = pd.concat([dfw0,dfw1,dfw2,dfw3], axis=0)
df_winter.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
6,Winter2017,11709,CO 0246,01,3.000,Family Communication,"[{'days': 'MTWRF', 'times': '0830am-0430pm', '...",['MTWRF 0830am-0430pm 01/04-01/10 DMH 350'],Margaret A. Wills,15,15,0
2,Winter2018,11827,BI 0071,OL1,3.000,Identity and the Human Genome,[],[],Christine R. Rodriguez,20,13,7
20,Winter2016,11620,NS 0687,01,1.000-6.000,DNP Immersion,[],[],Joyce Muriel Shea,20,4,16
23,Winter2018,11839,PY 0131,OL1,3.000,Abnormal Psychology for Non-Majors,[],[],Margaret M. McClure,20,20,0
11,Winter2016,11632,EE 0352,JD,3.000,Digital Communications,[],[],Jeffrey N. Denenberg,0,0,0


In [20]:
df_winter.describe()

,crn,cap,act,rem
count,105.000000,105.000000,105.000000,105.000000
mean,11524.266667,17.133333,12.342857,4.790476
std,355.382575,7.612069,6.584990,6.207463
min,10893.000000,0.000000,0.000000,-3.000000
25%,11603.000000,15.000000,9.000000,0.000000
50%,11633.000000,20.000000,12.000000,4.000000
75%,11802.000000,20.000000,18.000000,9.000000
max,11841.000000,30.000000,25.000000,27.000000


In [21]:
df_course = pd.concat([df_fall, df_spring, df_summer, df_winter], axis=0)
df_course.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
191,Spring2018,35056,BB 0132,01,0.000,Insurance Planning and Risk Management,"[{'days': 'T', 'times': '0630pm-1000pm', 'date...",['T 0630pm-1000pm 03/06-05/15 DMH 347'],Michael D. Strouch,15,3,12
70,Spring2019,37847,AE 0284,01,3.000,Environmental Ethics,"[{'days': 'F', 'times': '0200pm-0430pm', 'date...",['F 0200pm-0430pm 01/22-04/30 CNS 9'],Toby J. Svoboda,30,9,21
948,Fall2014,70200,MA 0371,A,3.000,Real Analysis,"[{'days': 'MWR', 'times': '0200pm-0250pm', 'da...",['MWR 0200pm-0250pm 09/02-12/08 BNW 127'],Irene T. Mulvey,18,18,0
784,Fall2016,77324,IT 0210,01,3.000,Intermediate Italian I,"[{'days': 'MWR', 'times': '0800am-0850am', 'da...",['MWR 0800am-0850am 09/06-12/12 DSB 2109A'],Joseph Vincent H. Morrissette,18,11,7
1195,Spring2015,33755,PY 0262,A,3.000,Sensation and Perception,"[{'days': 'MR', 'times': '0800am-0915am', 'dat...",['MR 0800am-0915am 01/20-04/30 BNW 131'],Michael Weiss,25,26,-1


In [22]:
df_course.describe()

,crn,cap,act,rem
count,15937.000000,15937.000000,15937.000000,15937.000000
mean,55172.299178,17.992784,15.578591,2.477317
std,19308.879968,12.671906,10.107690,12.709781
min,10893.000000,0.000000,-1.000000,-75.000000
25%,35426.000000,10.000000,7.000000,0.000000
50%,57145.000000,19.000000,15.000000,1.000000
75%,74541.000000,25.000000,23.000000,5.000000
max,79215.000000,500.000000,90.000000,999.000000


In [23]:
df_course.to_sql('import_course', conn, if_exists='append', index=False)

In [24]:
%%sql
SELECT *
FROM import_course
LIMIT 3;

 * sqlite:///CourseData2.db
Done.


term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
Fall2014,70384,AC 0011,C01,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31
Fall2014,70385,AC 0011,C02,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0930am-1045am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0930am-1045am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31
Fall2014,70382,AC 0011,C03,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 1230pm-0145pm 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31


**Creating dataframes for the import course_meeting**

In [25]:
cm_fall = ['SourceData/Fall2014/course_meetings.csv', 'SourceData/Fall2015/course_meetings.csv',
           'SourceData/Fall2016/course_meetings.csv', 'SourceData/Fall2017/course_meetings.csv',
          'SourceData/Fall2018/course_meetings.csv']
dfcmf1 = pd.read_csv(cm_fall[0])
dfcmf2  = pd.read_csv(cm_fall[1])
dfcmf3  = pd.read_csv(cm_fall[2])
dfcmf4  = pd.read_csv(cm_fall[3])
dfcmf5  = pd.read_csv(cm_fall[4])

In [26]:
df_cmfall = pd.concat([dfcmf1, dfcmf2, dfcmf3, dfcmf4, dfcmf5], axis=0)
df_cmfall.sample(5)

,term,crn,location,day,start,end
11234,Fall2014,70148,XVR 5,M,2014-11-24T13:00:00,2014-11-24T13:50:00
14459,Fall2016,77177,DSB 107,R,2016-12-01T09:30:00,2016-12-01T10:45:00
16138,Fall2017,71154,DSB 115,F,2017-11-03T09:30:00,2017-11-03T10:45:00
1923,Fall2016,77197,BNW 332,M,2016-10-17T08:00:00,2016-10-17T09:15:00
29393,Fall2016,78473,BNW 335,W,2016-11-30T16:00:00,2016-11-30T16:50:00


In [27]:
cm_spring = ['SourceData/Spring2015/course_meetings.csv',
           'SourceData/Spring2016/course_meetings.csv', 'SourceData/Spring2017/course_meetings.csv',
          'SourceData/Spring2018/course_meetings.csv', 'SourceData/Spring2019/course_meetings.csv']
dfcms1 = pd.read_csv(cm_spring[0])
dfcms2  = pd.read_csv(cm_spring[1])
dfcms3  = pd.read_csv(cm_spring[2])
dfcms4  = pd.read_csv(cm_spring[3])
dfcms5  = pd.read_csv(cm_spring[4])


In [28]:
df_cmspring = pd.concat([dfcms1, dfcms2, dfcms3, dfcms4, dfcms5], axis=0)
df_cmspring.sample(5)

,term,crn,location,day,start,end
15956,Spring2016,38795,DSB 111,W,2016-04-13T11:00:00,2016-04-13T13:30:00
1407,Spring2019,37154,DMH 347,M,2019-02-04T17:00:00,2019-02-04T18:15:00
5427,Spring2018,34161,DMH 348,T,2018-03-27T09:00:00,2018-03-27T09:50:00
20101,Spring2017,31263,MCA 102,F,2017-02-17T12:30:00,2017-02-17T13:45:00
17736,Spring2018,35924,BNW 131,M,2018-03-26T14:00:00,2018-03-26T14:50:00


In [29]:
cm_springbreak = ['SourceData/SpringBreak2017/course_meetings.csv']

dfcmsb1 = pd.read_csv(cm_springbreak[0])


In [30]:
cm_summer = ['SourceData/Summer2015/course_meetings.csv',
           'SourceData/Summer2016/course_meetings.csv', 'SourceData/Summer2017/course_meetings.csv',
          'SourceData/Summer2018/course_meetings.csv']
dfcmsu1 = pd.read_csv(cm_summer[0])
dfcmsu2  = pd.read_csv(cm_summer[1])
dfcmsu3  = pd.read_csv(cm_summer[2])
dfcmsu4  = pd.read_csv(cm_summer[3])

In [31]:
df_cmsummer = pd.concat([dfcmsu1, dfcmsu2, dfcmsu3, dfcmsu4], axis=0)
df_cmspring.sample(5)

,term,crn,location,day,start,end
25910,Spring2019,37107,BNW 340,W,2019-05-08T18:30:00,2019-05-08T21:00:00
11113,Spring2017,32672,DSB 109,F,2017-04-07T12:30:00,2017-04-07T13:45:00
21426,Spring2018,34507,MCA 206,M,2018-01-22T12:30:00,2018-01-22T13:45:00
16544,Spring2019,37842,BNW 334,T,2019-02-12T11:00:00,2019-02-12T12:15:00
3044,Spring2018,34367,BNW 269,W,2018-01-24T14:00:00,2018-01-24T16:50:00


In [32]:
cm_winter = ['SourceData/Winter2015/course_meetings.csv',
           'SourceData/Winter2016/course_meetings.csv', 'SourceData/Winter2017/course_meetings.csv',
          'SourceData/Winter2018/course_meetings.csv']
dfcmw1 = pd.read_csv(cm_winter[0])
dfcmw2  = pd.read_csv(cm_winter[1])
dfcmw3  = pd.read_csv(cm_winter[2])
dfcmw4  = pd.read_csv(cm_winter[3])

In [33]:
df_cmwinter = pd.concat([dfcmw1, dfcmw2, dfcmw3, dfcmw4], axis=0)
df_cmwinter.sample(5)

,term,crn,location,day,start,end
155,Winter2017,11721,CNS 15,S,2017-01-07T09:00:00,2017-01-07T15:00:00
49,Winter2015,10912,DSB 107,S,2015-01-17T08:30:00,2015-01-17T16:30:00
60,Winter2018,11833,DSB 109,T,2018-01-02T18:00:00,2018-01-02T21:30:00
58,Winter2015,10909,DSB 111,M,2015-01-05T08:30:00,2015-01-05T16:30:00
37,Winter2015,10912,DSB 107,T,2015-01-13T18:00:00,2015-01-13T21:30:00


In [34]:
df_coursemeeting = pd.concat([df_cmfall,df_cmspring,dfcmsb1,df_cmsummer,df_cmwinter],axis=0)

In [35]:
df_coursemeeting.to_sql('import_course_meeting', conn, if_exists='append', index=False)

In [36]:
%%sql
SELECT *
FROM import_course_meeting
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


term,crn,location,day,start,end
Fall2014,70384,DSB 105,T,2014-09-02T08:00:00,2014-09-02T09:15:00
Fall2014,70384,DSB 105,F,2014-09-05T08:00:00,2014-09-05T09:15:00
Fall2014,70384,DSB 105,T,2014-09-09T08:00:00,2014-09-09T09:15:00
Fall2014,70384,DSB 105,F,2014-09-12T08:00:00,2014-09-12T09:15:00
Fall2014,70384,DSB 105,T,2014-09-16T08:00:00,2014-09-16T09:15:00


**Creating the dataframes for import_course_catalog**

In [37]:
cc2017_2018 = ['SourceData/Catalogs/CourseCatalog2017_2018.csv']
dfcc17_18 = pd.read_csv(cc2017_2018[0])
dfcc17_18.sample(5)

,program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
2021,ENW,English: Writing,ENW 0498,Rhetoric and Composition Practicum,3 Credits,NaN,NaN,NaN,NaN,NaN
282,TX,Taxation,TX 0522,Taxation of Trusts and Estates,3 Credits,NaN,NaN,NaN,NaN,This course introduces students to the concept...
2033,NS,Nursing,NS 0666A,Practicum I: PMHNP,4 Credits,NS 0652*.,NaN,NaN,NaN,Practica experiences give students the opportu...
1961,CO,Communication,CO 0321,Communication Processes in Organizations: Nego...,3 Credits,CO 0220; junior or senior standing.,NaN,NaN,EVAP Environmental Studies: Applied Profession...,"This course reviews and explores, through simu..."
1241,RS,Religious Studies,RS 0289,Tantrism,3 Credits,RS 0101.,NaN,NaN,WDIV World Diversity,The course covers the medieval formation of ta...


In [38]:
cc2018_2019 = ['SourceData/Catalogs/CourseCatalog2018_2019.csv']
dfcc18_19 = pd.read_csv(cc2018_2019[0])
dfcc18_19.sample(5)

,program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
1986,FTM,"Film, Television, and Media Arts",FTM 0237,Acting for the Camera,3 Credits,TA 0030.,NaN,NaN,VPC2 Visual and Performing Arts Core: Applied ...,This course is an experiential introduction to...
1037,PH,Philosophy,PH 0316,Bergson,3 Credits,Two courses in Philosophy.,NaN,NaN,FREN French Course Taught in English,This course explores the work of Henri Bergson...
463,PS,Physics,PS 0071,Physics of Light and Color,3 Credits,NaN,NaN,NaN,EDCG Educational Studies Cognate,"This course, intended for students who are not..."
1461,ED,Education,ED 0329,Philosophy of Education: An Introduction,3 Credits,NaN,NaN,NaN,NaN,This course applies the basic concepts of phil...
165,AH,Art History,AH 0221,"The Arts of Ireland and the British Isles, 500...",3 Credits,One 100-level or lower art history course or p...,NaN,NaN,"CAOT Catholic Studies: Non-Religious Studies, ...",This course explores the art and architecture ...


In [39]:
df_catalog = pd.concat([dfcc17_18,dfcc18_19], axis=0)
df_catalog.sample(5)

,program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
1066,ME,Mechanical Engineering,ME 0349,Heat Transfer,3 Credits,ME 0347.,NaN,NaN,NaN,This course covers one- and two-dimensional he...
1380,ECE,Electrical and Computer Engineering,ECE 0448,Embedded Microcontrollers,3 Credits,NaN,NaN,NaN,NaN,Introduction to embedded microcontrollers in e...
617,RS,Religious Studies,RS 0205,Women in the Bible,3 Credits,RS 0101.,NaN,NaN,"JST Judaic Studies Minor, WSGF Women, Gender, ...",This course examines stories about women that ...
585,SW,Software Engineering,SW 0531,Applications and Data Security,3 Credits,NaN,NaN,NaN,NaN,This course is structured around enterprise an...
1600,CL,Classical Studies,CL 0108,Myth in Classical Literature,3 Credits,NaN,NaN,NaN,NaN,This course introduces students to classical m...


In [40]:
df_catalog.to_sql('import_course_catalog', conn, if_exists='append', index=False)

In [41]:
%%sql
SELECT *
FROM import_course_catalog
LIMIT 10;

 * sqlite:///CourseData2.db
Done.


program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
AN,Asian Studies,AN 0301,Independent Study,1-3 Credits,None,None,None,None,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.
AN,Asian Studies,AN 0310,Asian Studies Seminar,3 Credits,None,None,None,None,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only."
BU,Business,BU 0211,Legal Environment of Business,3 Credits,Junior standing.,None,None,None,"This course examines the broad philosophical as well as practical nature and function of the legal system, and introduces students to the legal and social responsibilities of business. The course includes an introduction to the legal system, the federal courts, Constitutional law, the United States Supreme Court, the civil process, and regulatory areas such as employment discrimination, protection of the environment, and corporate governance and securities markets."
BU,Business,BU 0220,Environmental Law and Policy,3 Credits,None,None,None,"EVME Environmental Studies Major Elective, EVPE Environmental Studies Elective, EVSS Environmental Studies: Social Science, MGEL Management: General Elective","This course surveys issues arising out of federal laws designed to protect the environment and manage resources. It considers in detail the role of the Environmental Protection Agency in the enforcement of environmental policies arising out of such laws as the National Environmental Policy Act, the Clean Water Act, and the Clear Air Act, among others. The course also considers the impact of Congress, political parties, bureaucracy, and interest groups in shaping environmental policy, giving special attention to the impact of environmental regulation on business and private property rights."
BU,Business,BU 0311,"The Law of Contracts, Sales, and Property",3 Credits,BU 0211.,None,None,None,"This course examines the components of common law contracts including the concepts of offer and acceptance, consideration, capacity and legality, assignment of rights and delegation of duties, as well as discharge of contracts. The course covers Articles 2 and 2A of the Uniform Commercial Code relating to leases, sales of goods, and warranties. The course also considers personal and real property, and bailments."
BU,Business,BU 0312,The Law of Business Organizations and Financial Transactions,3 Credits,BU 0211.,None,None,None,"This course offers an analysis of legal principles related to the law of agency, sole proprietorships, partnerships, corporations, limited liability companies, and other business forms. The second half of the course addresses several sections of the Uniform Commercial Code, such as negotiable instruments, bank collections and deposits and secured transactions. Finally, the course examines the law of suretyship, debtor-creditor relationships, and bankruptcy."
BU,Business,BU 0320,Employment Law and Discrimination in the Workplace,3 Credits,None,None,None,"MGEL Management: General Elective, UDIV U.S. Diversity","This course examines a variety of legal issues related to the workplace including the doctrine of employment at will, employee privacy, and the history and development of labor unions and the legal protections afforded by the National Labor Relations Act. A study of the

## Populate ERD Tables with data from Import Tables ##

In [42]:
%%sql

--Populating Course_Meeting table using import_course_meeting data-- 

INSERT INTO Course_Meeting (CRN, Term, Location, Start, End, Day)
SELECT DISTINCT crn, term, location, start, end, day
FROM import_course_meeting;

 * sqlite:///CourseData2.db
Done.


[]

In [43]:
%%sql 

--Checking data population of Course_Meeting table-- 

SELECT * 
FROM Course_Meeting
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


MID,CRN,Term,Location,Start,End,DAY
1,70384,Fall2014,DSB 105,2014-09-02T08:00:00,2014-09-02T09:15:00,T
2,70384,Fall2014,DSB 105,2014-09-05T08:00:00,2014-09-05T09:15:00,F
3,70384,Fall2014,DSB 105,2014-09-09T08:00:00,2014-09-09T09:15:00,T
4,70384,Fall2014,DSB 105,2014-09-12T08:00:00,2014-09-12T09:15:00,F
5,70384,Fall2014,DSB 105,2014-09-16T08:00:00,2014-09-16T09:15:00,T


In [44]:
%%sql

--Populating PROGRAM table using import_course_catalog data-- 

INSERT INTO PROGRAM(PName, PCode)
SELECT DISTINCT program_name, program_code
FROM import_course_catalog;

 * sqlite:///CourseData2.db
Done.


[]

In [45]:
%%sql 

SELECT * 
FROM PROGRAM
LIMIT 10;

 * sqlite:///CourseData2.db
Done.


PID,PName,PCode
1,Asian Studies,AN
2,Business,BU
3,Black Studies,BL
4,Bioengineering,BEN
5,Arabic,AR
6,Anthropology,AY
7,American Studies,AS
8,Japanese,JA
9,Art History,AH
10,Applied Ethics,AE


In [46]:
%%sql 

--Populating Catalog table using import_course_catalog data--  

INSERT INTO CATALOG (CatID, Title, Description, Credits, PID, PREREQS, COREQS, FEES, Attribute)
SELECT DISTINCT catalog_id, course_title, description, credits, PID, prereqs, coreqs, fees, attributes
FROM import_course_catalog
    JOIN PROGRAM ON (import_course_catalog.program_name = program.PName);

 * sqlite:///CourseData2.db
Done.


[]

In [47]:
%%sql 

--Checking data population of Catalog table-- 

SELECT * 
FROM CATALOG
LIMIT 2;

 * sqlite:///CourseData2.db
Done.


CatClass,CatID,Title,Description,Credits,PID,PREREQS,COREQS,FEES,Attribute
1,AN 0301,Independent Study,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.,1-3 Credits,1,None,None,None,None
2,AN 0310,Asian Studies Seminar,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only.",3 Credits,1,None,None,None,None


In [48]:
%%sql 

--Populating Instructor table using import_courses data--  

INSERT INTO Instructor (Name)
SELECT DISTINCT primary_instructor
FROM import_course
WHERE primary_instructor <> 'TBA' AND primary_instructor NOT LIKE 'primary_instructor' AND primary_instructor NOT LIKE '%/';

 * sqlite:///CourseData2.db
Done.


[]

In [49]:
%%sql 

--Checking data population of Instructor table-- 

SELECT * 
FROM Instructor
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


InID,Name
1,Michael P. Coyne
2,Rebecca I. Bloch
3,Paul Caster
4,Jo Ann Drusbosky
5,Arleen N. Kardos


In [50]:
%%sql 

--Populating the SECTION table using import_course data--
--Joining Instructor table using import_course and instructor table--

INSERT INTO SECTION (CRN, Term, Section, Cap, Act, Rem, CatID, InID)
SELECT crn, term, section, cap, act, rem, catalog_id, InID
FROM import_course
    JOIN Instructor ON (import_course.primary_instructor = Instructor.Name)
    JOIN Catalog ON (import_course.catalog_id = Catalog.CatID)
;

 * sqlite:///CourseData2.db
Done.


[]

In [51]:
%%sql 

--Checking data population of Section table--

SELECT * 
FROM Section
LIMIT 30;

 * sqlite:///CourseData2.db
Done.


SID,CRN,Term,Section,Cap,Act,Rem,CatID,InID
1,70384,Fall2014,C01,0,31,-31,AC 0011,1
2,70385,Fall2014,C02,0,31,-31,AC 0011,1
3,70382,Fall2014,C03,0,31,-31,AC 0011,1
4,70291,Fall2014,C04,0,29,-29,AC 0011,2
5,70350,Fall2014,C05,0,30,-30,AC 0011,2
6,70381,Fall2014,C06,0,31,-31,AC 0011,2
7,70383,Fall2014,G,30,31,-1,AC 0011,3
8,70391,Fall2014,H,30,32,-2,AC 0011,4
9,71105,Fall2014,I,30,33,-3,AC 0011,4
10,71123,Fall2014,J,30,32,-2,AC 0011,4


## Drop Import Tables ##

In [52]:
%%sql
--Drop import tables--
DROP TABLE IF EXISTS import_course;
DROP TABLE IF EXISTS import_course_meeting;
DROP TABLE IF EXISTS import_course_catalog;

 * sqlite:///CourseData2.db
Done.
Done.
Done.


[]

In [53]:
%%sql
DROP TABLE IF EXISTS Instructor_dw;
DROP TABLE IF EXISTS COURSE;
DROP TABLE IF EXISTS MEETING;

 * sqlite:///CourseData2.db
Done.
Done.
Done.


[]